In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypertension"
cohort = "GSE151158"

# Input paths
in_trait_dir = "../../input/GEO/Hypertension"
in_cohort_dir = "../../input/GEO/Hypertension/GSE151158"

# Output paths
out_data_file = "../../output/preprocess/Hypertension/GSE151158.csv"
out_gene_data_file = "../../output/preprocess/Hypertension/gene_data/GSE151158.csv"
out_clinical_data_file = "../../output/preprocess/Hypertension/clinical_data/GSE151158.csv"
json_path = "../../output/preprocess/Hypertension/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional analysis of non-fibrotic NAFLD progression"
!Series_summary	"Background & Aims:  Non-alcoholic steatohepatitis (NASH), a subtype of non-alcoholic fatty liver disease (NAFLD) that can lead to fibrosis, cirrhosis, and hepatocellular carcinoma, is characterized by hepatic inflammation.  Despite evolving therapies aimed to ameliorate inflammation in NASH, the transcriptional changes that lead to inflammation progression in NAFLD remain poorly understood.  The aim of this study is to define transcriptional changes in early, non-fibrotic NAFLD using a biopsy-proven non-fibrotic NAFLD cohort.  Methods:  We extracted RNA from liver tissue of 40 patients with biopsy-proven NAFLD based on NAFLD Activity Score (NAS) (23 with NAS ≤3, 17 with NAS ≥5) and 21 healthy controls and compared changes in expression of 594 genes involved in innate immune function.    Results:  Compared to healthy controls, NAFLD patients with NAS ≥5 had differential ex

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data for liver tissue
# from patients with NAFLD and healthy controls.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For hypertension (trait), row 7 contains hypertension data
trait_row = 7

# For age, row 1 contains age data
age_row = 1

# For gender, row 2 contains sex data
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert hypertension data to binary (0=N, 1=Y)."""
    if pd.isna(value):
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'Y':
        return 1
    elif value.upper() == 'N':
        return 0
    return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if pd.isna(value):
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0=F, 1=M)."""
    if pd.isna(value):
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    return None

# 3. Save Metadata
# Initial filtering on the usability of the dataset based on gene and trait availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Find the clinical data file(s) in the cohort directory
    import glob
    import os
    
    clinical_files = glob.glob(os.path.join(in_cohort_dir, "*_clinical.csv"))
    if clinical_files:
        clinical_data = pd.read_csv(clinical_files[0])
        
        # Extract clinical features using the function from tools.preprocess
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Clinical features preview:", preview)
        
        # Save the extracted clinical features to a CSV file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP',
       'AREG', 'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5',
       'ATG7', 'ATM', 'B2M'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown in the output, these appear to be standard human gene symbols.
# For example:
# - ABCB1 is the gene encoding ATP Binding Cassette Subfamily B Member 1
# - ABL1 is the ABL Proto-Oncogene 1, Non-Receptor Tyrosine Kinase
# - B2M is Beta-2-Microglobulin
# - ATM is Ataxia Telangiectasia Mutated

# These are already in the preferred format of human gene symbols, so no mapping is needed.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the function from tools.preprocess
# Use the parameters defined in STEP 2 for the trait, age, and gender rows
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the extracted clinical features to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data before missing value handling: {linked_data.shape}")

# 4. Handle missing values systematically
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after missing value handling: {linked_data.shape}")

# 5. Check if the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort information
note = "Dataset from NAFLD study with hypertension trait information available."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to quality issues.")

Normalized gene data saved to ../../output/preprocess/Hypertension/gene_data/GSE151158.csv
Clinical data saved to ../../output/preprocess/Hypertension/clinical_data/GSE151158.csv
Shape of linked data before missing value handling: (66, 586)
Shape of linked data after missing value handling: (61, 586)
For the feature 'Hypertension', the least common label is '1.0' with 20 occurrences. This represents 32.79% of the dataset.
The distribution of the feature 'Hypertension' in this dataset is fine.



Quartiles for 'Age':
  25%: 37.0
  50% (Median): 45.0
  75%: 52.0
Min: 15.0
Max: 65.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 25 occurrences. This represents 40.98% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Linked data saved to ../../output/preprocess/Hypertension/GSE151158.csv
